In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint    
import seaborn as sns
import datetime 
%matplotlib inline

# Pull Data

In [2]:
path = "C:/Users/Akis-/OneDrive/Masters/Web Economics/Data/"
#path = "C:/temp/kaggle/webeconomics/"
#test_df = pd.read_csv(path+'test.csv')
#train_df = pd.read_csv(path+'train.csv')
validation_df = pd.read_csv(path+'validation.csv')
#train_df = pd.read_csv(path+'validation.csv')

In [3]:
#TotalImpressions = test_df.shape[0]
#print("TotalImpressions:",TotalImpressions)

In [4]:
#budget=6250*1000
#print("Our Budget is:",budget, "Fen")
#print("To use over",test_df.shape[0],"bid opportunities.")
#BudPerBid=budget/test_df.shape[0]
#print("Budget / Bid Opportunity:",BudPerBid)

minBid=np.min(train_df["payprice"].values)
maxBid=np.max(train_df["payprice"].values)

step_size = 1
custRange = np.arange(minBid+1, maxBid+5,step_size) # determines the range that bids should be in
#custRange = np.arange(60, 65,1)

minCustRange = np.arange(21, 91, step_size) # determines the range that bids should be in
maxCustRange = np.arange(71, 161, step_size) # determines the range that bids should be in

NameError: name 'train_df' is not defined

In [5]:
pd.set_option('display.max_columns', None)

# Functions

### Plot Metrics for bound combination for random bidding strategy

In [6]:
def plotResults(Matrix):

    ax = sns.heatmap(Matrix, linewidth=0, xticklabels=minCustRange, yticklabels=maxCustRange[::-1], cmap="Greens") #, annot=True
    ax.set(xlabel='lower bound for random bid', ylabel='upper bound for random bid', title="Clicks by bounded random bids")
    for label in ax.xaxis.get_ticklabels()[::1]:
        label.set_visible(False)
    for label in ax.xaxis.get_ticklabels()[::5]:
        label.set_visible(True)
    for label in ax.yaxis.get_ticklabels()[::1]:
        label.set_visible(False)
    for label in ax.yaxis.get_ticklabels()[::5]:
        label.set_visible(True)
        
#     for label in ax.xaxis.get_ticklabels()[::2]:
#         label.set_visible(False)    
#     for label in ax.yaxis.get_ticklabels()[::2]:
#         label.set_visible(False)    
        
        
    plt.savefig('RandomBidResults.png')
    plt.show()
    
    useless = 0
    return useless

### Build Matrix containing CTR for various bound combinations

In [7]:
def BuildCTRMatrix(dataframe,plot,repeats):
    #lastInRange = custRange[len(custRange)-1:][0]+1
    for n in range(0,repeats,1):
        dataSubset = dataframe.sample(frac=0.5, replace=False)
        print(n+1,"/",repeats,"...")
        
        for minimumBid in minCustRange:
            print('counting clicks',minimumBid,datetime.datetime.now().time())        
            for maximumBid in maxCustRange: #maxBid>minBid?????
                if maximumBid>minimumBid:
                    Clicks = EvalRandBidClicksOnly(dataSubset,minimumBid, maximumBid,6250000,validation_df.shape[0])
                    CTRMatrix[minimumBid][maximumBid]=(CTRMatrix[minimumBid][maximumBid]*n+Clicks)/(n+1)
        CTRMatrix.to_csv("RandomBidResults.csv")
    return CTRMatrix

### Extract best Bounds from Results Matrix

In [8]:
def FindBestBounds(ResultsMatrix):
    print("Best CTR is:",ResultsMatrix.values.max())
    for x in minCustRange:
        for y in maxCustRange:
            if ResultsMatrix[x][y]==ResultsMatrix.values.max():
                print("X:",x,"Y:",y)
                lowerBound=x
                upperBound=y
    return lowerBound, upperBound

### Evaluate a random bid value on a new set

In [9]:
#def EvalRandBid(dataframe,lowerBound, upperBound):
#    
#    dataframe['ConstBid'] = np.random.randint(lowerBound, upperBound+1, dataframe.shape[0])
#
#    trueValues = (dataframe["payprice"]<dataframe['ConstBid'])
#    impressions = dataframe[trueValues].shape[0]
#    clicks = np.sum(dataframe[trueValues]["click"].values)
#    ClickThroughRate=(clicks*100)/impressions
#        
#    return ClickThroughRate

In [10]:
def EvalRandBidClicksOnly(dataframe,lowerBound, upperBound,budget,size):
    
    AdjustedBudget=(budget/size)*dataframe.shape[0]
    #print("AdjustedBudget is:",AdjustedBudget)
    tempData=dataframe
    #tempData['ConstBid']=constant
    tempData['ConstBid'] = np.random.randint(lowerBound, upperBound+1, dataframe.shape[0])

    tempData['trueValues'] = np.where(tempData["payprice"]<tempData['ConstBid'],1,0)
    tempData['ModelPays'] = tempData['trueValues']*tempData['payprice']
    tempData['cumsum'] = tempData['ModelPays'].cumsum()
    lastRowToInclude = np.argmax(tempData['cumsum'].as_matrix()>AdjustedBudget)
    if lastRowToInclude==0:
        lastRowToInclude=tempData.shape[0]
    #print("lastRowToInclude",lastRowToInclude)
    
    shortData = tempData.head(lastRowToInclude)
    trueValues = (0<shortData['ModelPays'])
    ##impressions = shortData[trueValues].shape[0]
    clicks = np.sum(shortData[trueValues]["click"].values)
    ##spend = np.sum(shortData[trueValues]["payprice"].values)
    ##CostPerClick = spend/clicks
    ##CostPerMille = spend*1000/(impressions)
    ##ClickThroughRate=(clicks*100)/impressions
    #print(constant, ClickThroughRate, clicks, spend, CostPerMille, CostPerClick, impressions)
    #return [ClickThroughRate, clicks, spend, CostPerMille, CostPerClick, impressions]
    return clicks


    #impressions = shortData.loc[shortData['ModelPays'] > 0].sum()
    #clicks = shortData.loc[shortData['ModelPays'] > 0 , 'click'].sum()
    #print("clicks:",clicks)

In [11]:
def EvalRandBid(dataframe,lowerBound, upperBound,budget,size):
    
    AdjustedBudget=(budget/size)*dataframe.shape[0]
    #print("AdjustedBudget is:",AdjustedBudget)
    tempData=dataframe
    #tempData['ConstBid']=constant
    tempData['ConstBid'] = np.random.randint(lowerBound, upperBound+1, dataframe.shape[0])

    tempData['trueValues'] = np.where(tempData["payprice"]<tempData['ConstBid'],1,0)
    tempData['ModelPays'] = tempData['trueValues']*tempData['payprice']
    tempData['cumsum'] = tempData['ModelPays'].cumsum()
    lastRowToInclude = np.argmax(tempData['cumsum'].as_matrix()>AdjustedBudget)
    if lastRowToInclude==0:
        lastRowToInclude=tempData.shape[0]
    #print("lastRowToInclude",lastRowToInclude)
    
    shortData = tempData.head(lastRowToInclude).copy(True)
    trueValues = (0<shortData['ModelPays'])
    impressions = shortData[trueValues].shape[0]
    clicks = np.sum(shortData[trueValues]["click"].values)
    spend = np.sum(shortData[trueValues]["payprice"].values)
    CostPerClick = spend/clicks
    CostPerMille = spend*1000/(impressions)
    ClickThroughRate=(clicks*100)/impressions
    #print(constant, ClickThroughRate, clicks, spend, CostPerMille, CostPerClick, impressions)
    return [ClickThroughRate, clicks, spend, CostPerMille, CostPerClick, impressions]
    #return clicks


    #impressions = shortData.loc[shortData['ModelPays'] > 0].sum()
    #clicks = shortData.loc[shortData['ModelPays'] > 0 , 'click'].sum()
    #print("clicks:",clicks)

# Main Code

### Determine Best Bounds Combination from Train Set

In [ ]:
zero_data = np.zeros(shape=(len(custRange),len(custRange)))
zero_data[zero_data < 1] = 0.0
CTRMatrix=pd.DataFrame(zero_data,index=reversed(custRange), columns=custRange)

In [ ]:
CTRMatrix =  BuildCTRMatrix(train_df,plot=False,repeats=5)
#CTRMatrix.to_pickle("CTRMatrix.pkl")

In [ ]:
CTRMatrix.to_csv("RandomBidResults.csv")

In [ ]:
#CTRMatrix = pd.read_pickle("CTRMatrix.pkl")

In [ ]:
plotResults(CTRMatrix)

In [ ]:
#CTRMatrix

print(minCustRange)

print(maxCustRange)

t = CTRMatrix.iloc[(304-160):(304-70),20:90]

plotResults(t)

In [ ]:
lowBound, upBound = FindBestBounds(t)

### Evalute on a new data set (validation set)

In [12]:
result = EvalRandBid(validation_df,69, 89,6250000,validation_df.shape[0])

In [13]:
#constant = result[0]
ClickThroughRate = result[0]
clicks = result[1]
spend = result[2]
CostPerMille = result[3]
CostPerClick = result[4]
impressions = result[5]

#print("constant",constant)
print("ClickThroughRate",ClickThroughRate)
print("clicks",clicks)
print("spend",spend)
print("CostPerMille",CostPerMille)
print("CostPerClick",CostPerClick)
print("impressions",impressions)

ClickThroughRate 0.043532676715459545
clicks 64
spend 6249934
CostPerMille 42511.93067421233
CostPerClick 97655.21875
impressions 147016
